# Test

Agreement and policy: https://developer.twitter.com/en/developer-terms/agreement-and-policy

In [1]:
import os
os.chdir('..')

import requests
import tweepy
import pandas as pd
from modules import TwitterAuthentication

twitter = TwitterAuthentication()
client = tweepy.Client(bearer_token=twitter.bearer_token)

## Tweepy example

¿En qué necesito pensar?

- ¿Qué campos necesito? Listo.
- ¿Cómo expandir public metrics?
- Esquema para extraer datos uniformemente durante los últimos 7 días
- Establecer qué políticos tomar en cuenta. Scrape the list from here: https://es.wikipedia.org/wiki/Elecciones_generales_de_Guatemala_de_2023
- ¿Guardar los datos en una base de datos SQL local?


In [2]:
def get_tweets(query: str, start_time: str, end_time: str, max_results: int) -> requests.Response:
    query = query = f"{query} -is:retweet -is:reply"
    tweets = client.search_recent_tweets(
        query=query,
        start_time=start_time,
        end_time=end_time,
        max_results=max_results,
        tweet_fields=[
            "author_id", "created_at", "text", "entities", 
            "geo", "id", "lang", "public_metrics",
            "possibly_sensitive", "referenced_tweets"
        ],
        user_fields=[
            "username", "name", "created_at", "description", "id", 
            "entities", "location", "pinned_tweet_id", "profile_image_url", 
            "public_metrics", "verified", "withheld"
        ],
        expansions=[
            "author_id", "referenced_tweets.id"
        ]
    )
    return tweets

In [6]:
# Prepare lists to store tweet and user data
tweet_data = []
user_data = {}

# Extract tweet and user data
for tweet in tweets.data:
    tweet_dict = {key: getattr(tweet, key) for key in tweet.data.keys()}
    tweet_data.append(tweet_dict)
    author_id = tweet.author_id
    user = [u for u in tweets.includes['users'] if u.id == author_id][0]
    user_data[author_id] = {key: getattr(user, key) for key in user.data.keys()}

# Create DataFrames for tweets and users
tweets_df = pd.DataFrame(tweet_data)
users_df = pd.DataFrame(user_data.values())

# Merge DataFrames on author_id
df = tweets_df.merge(users_df, left_on='author_id', right_on='id', suffixes=('_tweet', '_user'))

df.to_csv('tweets.csv', index=False)

In [3]:
# Example usage
query = 'sammy morales'
start_time = "2023-04-25T00:00:00Z" 
end_time = "2023-04-28T00:00:00Z" 
max_results = 10

tweets = get_tweets(
    query=query,
    start_time=start_time,
    end_time=end_time,
    max_results=max_results,
)

In [4]:
tweets

Response(data=[<Tweet id=1651701981046513666 text='Sammy Morales sabe que con el Libramiento de Chimaltenango su hermano Jimmy Morales robó millones del erario nacional. https://t.co/E9ndjElmG4'>, <Tweet id=1651694114390392834 text='((AL AIRE)) 🔊\xa0En #Hora15 “La Hora de los Candidatos”. Seguimos con las propuestas políticas de los candidatos a la presidencia. El invitado en cabina es Sammy Morales, candidato del partido Frente de Convergencia Nacional (FCN-Nación). Participe 📷\n\n📻\xa0https://t.co/YuLQ5FB7OW https://t.co/vhKYzOHrO7 https://t.co/1YPjnH8rqb'>, <Tweet id=1651694055292583936 text='((AL AIRE)) 🔊 En #Hora15 “La Hora de los Candidatos”. Seguimos con las propuestas políticas de los candidatos a la presidencia. El invitado en cabina es Sammy Morales, candidato del partido Frente de Convergencia Nacional (FCN-Nación). Participe \n\n📻 https://t.co/KgLpSYC2pM https://t.co/NOmrj4lAuy'>, <Tweet id=1651660145456193556 text='Pero el hijueputamas decie que van por mas, con Manuel Con